In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pywt
import matplotlib.pyplot as plt

from mne import read_epochs, set_log_level, compute_rank, concatenate_epochs
from mrmr import mrmr_classif
from random import randint
from mne.decoding import Scaler
from scipy.stats import kurtosis, skew, moment, entropy, norm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, cross_val_score, GridSearchCV,ShuffleSplit
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('gruvboxd')

In [ ]:
set_log_level('warning')
epochs = read_epochs('ica_epo.fif').pick('eeg').filter(0,240)
epochs.drop_channels(epochs.info['bads'])
epochs.apply_baseline((-1.4,-0.4))

In [ ]:
hbo = read_epochs('hbo_epo.fif')
hbr = read_epochs('hbr_epo.fif')
hbo

In [ ]:
scaler = Scaler(info=epochs.info)
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
le = LabelEncoder()

In [ ]:
def dwt_det_coeff(x, db='db2'):
    aprx, det = pywt.dwt(x,db)
    return det

def dwt_aprox_coeff(x, db='db2'):
    aprx, det = pywt.dwt(x,db)
    return aprx

def rms(x):
    return np.sqrt(np.mean(x**2))

def slope(x):
    t = np.linspace(0, len(x)-1, len(x))
    return np.polyfit(t, x, 1)[0]

def autocorr(x):
    return float(np.correlate(x,x))
    
def temp_centroid(x):
    nom = [x[i] * i for i in range(x.shape[0])]
    return sum(nom)/sum(x)

def energy(x):
    return sum(x**2)

def med_abs_diff(x):
    return np.median(np.abs(np.diff(x)))

def mean_abs_diff(x):
    return np.mean(np.abs(np.diff(x)))

def calc_centroid(x, fs=7.81):
    energy = np.array(x) ** 2
    t = range(len(x))
    t = [float(x) / fs for x in t]
    t_energy = np.dot(np.array(t), np.array(energy))
    energy_sum = np.sum(energy)

    if energy_sum == 0 or t_energy == 0:
        centroid = 0
    else:
        centroid = t_energy / energy_sum

    return centroid

left rest

In [15]:
conditions = ['left','rest']
hbo_sub = hbo[conditions].copy()
hbr_sub = hbr[conditions].copy()

subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
eeg_data = subset.copy().crop(0.4,1.2).get_data()

hbo_sub.drop_channels(hbo_sub.info['bads'])
hbr_sub.drop_channels(hbr_sub.info['bads'])
y = le.fit_transform(hbr_sub.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

t_min = 1
t_max = 10

hbo_sub = hbo_sub.copy().crop(t_min,t_max).get_data()
hbr_sub = hbr_sub.copy().crop(t_min,t_max).get_data()
hbo_avg = np.mean(hbo_sub, axis=0, keepdims=True)
hbr_avg = np.mean(hbr_sub, axis=0, keepdims=True)

hbo_sub = hbo_sub - hbo_avg
hbr_sub = hbr_sub - hbr_avg

nirs_data = np.concatenate([hbo_sub,  hbr_sub], axis=1)

y = le.fit_transform(subset.events[:,2])


In [13]:
test_fnirs_eeg('bior2.2', 6, 5, 12)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.52it/s]


0.8308823529411764


In [16]:
test_fnirs_eeg('bior2.2', 3, 5, 11)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:06<00:00,  1.64it/s]


0.9257352941176471


0.9257352941176471

In [12]:
test_fnirs_eeg('bior2.2', 3, 5, 14)

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:06<00:00,  2.02it/s]


0.7823529411764706


0.7823529411764706

left vs right

In [ ]:
conditions = ['left','right']
hbo_sub = hbo[conditions].copy()
hbr_sub = hbr[conditions].copy()

subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
eeg_data = subset.copy().crop(0.2,1).get_data()

hbo_sub.drop_channels(hbo_sub.info['bads'])
hbr_sub.drop_channels(hbr_sub.info['bads'])
y = le.fit_transform(hbr_sub.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

t_min = 1
t_max = 9

hbo_sub = hbo_sub.copy().crop(t_min,t_max).get_data()
hbr_sub = hbr_sub.copy().crop(t_min,t_max).get_data()
hbo_avg = np.mean(hbo_sub, axis=0, keepdims=True)
hbr_avg = np.mean(hbr_sub, axis=0, keepdims=True)

hbo_sub = hbo_sub - hbo_avg
hbr_sub = hbr_sub - hbr_avg

nirs_data = np.concatenate([hbo_sub,  hbr_sub], axis=1)

y = le.fit_transform(subset.events[:,2])


In [9]:
test_n_features('db30', 4, 5)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.66it/s]


a: 0.8823529411764706 std 0.08318903308077032
0.8823529411764706 at  10 ft


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  3.00it/s]


a: 0.823529411764706 std 0.08318903308077026


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.59it/s]


...

a: 0.8627450980392156 std 0.027729677693590107


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:06<00:00,  2.59it/s]


a: 0.8431372549019608 std 0.13864838846795047


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.48it/s]


KeyboardInterrupt: 

In [29]:
selected_features, single_score = test_fnirs_eeg('db30', 4, 5, 15)
selected_features = np.concatenate(selected_features)
np.save('single_eeg_nirs_features', selected_features)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:03<00:00,  3.77it/s]


a: 0.8823529411764707 std 0.09112901991076273


Rhythmn

In [25]:
conditions = ['r_pinch','r_stop']
hbo_sub = hbo[conditions].copy().apply_baseline((-5,0))
hbr_sub = hbr[conditions].copy().apply_baseline((-5,0))

subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
eeg_data = subset.copy().crop(4.1,7.8).get_data()

hbo_sub.drop_channels(hbo_sub.info['bads'])
hbr_sub.drop_channels(hbr_sub.info['bads'])
y = le.fit_transform(hbr_sub.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

t_min = 6
t_max = 20

hbo_sub = hbo_sub.copy().crop(t_min,t_max).get_data()
hbr_sub = hbr_sub.copy().crop(t_min,t_max).get_data()
hbo_avg = np.mean(hbo_sub, axis=0, keepdims=True)
hbr_avg = np.mean(hbr_sub, axis=0, keepdims=True)

hbo_sub = hbo_sub - hbo_avg
hbr_sub = hbr_sub - hbr_avg

nirs_data = np.concatenate([hbo_sub,  hbr_sub], axis=1)

y = le.fit_transform(subset.events[:,2])


In [15]:
selected_features, rhythmn_score = test_fnirs_eeg('db30', 4, 5, 14)
selected_features = np.concatenate(selected_features)
np.save('rhythmn_eeg_nirs_features', selected_features)


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  2.91it/s]


a: 0.93125 std 0.06525191568069093


In [30]:
np.save('eeg_nirs_scores', np.array([single_score, rhythmn_score]))

In [ ]:
def test_n_features(db, lvl, lvl1, fourth=False):
    best_score = 0
    for n_features in np.arange(15,24):
        fts, current = test_fnirs_eeg(db,lvl, lvl1, fourth, n_features)
        if current > best_score:
            best_score = current
            print(best_score, 'at ', n_features, 'ft')
            
def get_feature_names(lvl, lvl1, fourth=False):
    all_features = []
    signal_features = ['std', 'max', 'min', 'rms', 'slope', 'skew', 'avg', 'mean_abs','temp_centr','enrgy','krts','st_centr']
    nirs_features = ['std', 'mean', 'min', 'max', 'slope', 'skew', 'krts','tmp_centr']
    
    rg = 4 if fourth else 3
    for level in range(rg):
        for ft in signal_features:
            for ch in subset.info['chs']:
                if level == 0:
                    all_features.append('D'+str(lvl) + '_' + ch['ch_name'] + '_' + ft)
                elif level == 1:
                    all_features.append('D'+str(lvl1) + '_' + ch['ch_name'] + '_' + ft)
                elif level == 3:
                    all_features.append('D'+str(fourth) + '_' + ch['ch_name'] + '_' + ft)
                else :
                    all_features.append('A0'+ '_' + ch['ch_name'] + '_' + ft)
        
        
    for ch in hbo.info['chs']:
        for ft in nirs_features:
            all_features.append(ch['ch_name'] + ft)

    for ch in hbr.info['chs']:
        for ft in nirs_features:
            all_features.append(ch['ch_name'] + ft)

    return all_features

def test_fnirs_eeg(db, lvl, lvl1,fourth=False, features=10):
    selected_features = []
    score = []
    ft_names = get_feature_names(lvl, lvl1, fourth) 

    for train_rep in range(6):
        cv = StratifiedKFold(n_splits=3,shuffle=True, random_state=randint(15,50))        
        cv_split = cv.split(eeg_data, y)
        
        mrmr_features = None
        
        median_score = []
        for train_idx, test_idx in cv_split:
            y_train, y_test = y[train_idx], y[test_idx]
            x_aprox_coeff = eeg_data[train_idx]
            test_aprox_coeff = eeg_data[test_idx]
            nirs_train = nirs_data[train_idx]
            nirs_test = nirs_data[test_idx]
            
            detail_coeffs = []
            aprox_coeffs = []
            test_detail_coeffs = []
            test_aprox_coeffs = []
            
            for dwt_lvl in range(lvl1+1):
                x_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, x_aprox_coeff, db=db)
                x_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, x_aprox_coeff, db=db)
                test_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, test_aprox_coeff, db=db)
                test_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, test_aprox_coeff, db=db)

                detail_coeffs.append(scaler.fit_transform(x_det_coeff.copy(),y_train))
                test_detail_coeffs.append(scaler.transform(test_det_coeff.copy()))
                aprox_coeffs.append(scaler.fit_transform(x_aprox_coeff.copy(),y_train))
                test_aprox_coeffs.append(scaler.transform(test_aprox_coeff.copy()))
                
            x_train = [] 
            
            x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl]**2))
            x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl]))
            x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl]))

            x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl1]**2))
            x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl1]))
            x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl1]))
            
            x_train.append(np.apply_along_axis(np.std, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(np.max, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(np.min, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(rms, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(slope, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(skew, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(np.average, 2, aprox_coeffs[-1]**2))
            x_train.append(np.apply_along_axis(mean_abs_diff, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(temp_centroid, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(energy, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(kurtosis, 2, aprox_coeffs[-1]))
            x_train.append(np.apply_along_axis(calc_centroid, 2, aprox_coeffs[-1]))

            if fourth :
                x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[fourth]**2))
                x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[fourth]))
                x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[fourth]))

            x_train.append(np.apply_along_axis(np.std, 2, nirs_data[train_idx]))
            x_train.append(np.apply_along_axis(np.mean, 2, nirs_data[train_idx]))
            x_train.append(np.apply_along_axis(np.min, 2, nirs_data[train_idx]))
            x_train.append(np.apply_along_axis(np.max, 2, nirs_data[train_idx]))
            x_train.append(np.apply_along_axis(slope, 2, nirs_data[train_idx]))
            x_train.append(np.apply_along_axis(skew, 2, nirs_data[train_idx]))
            x_train.append(np.apply_along_axis(kurtosis, 2, nirs_data[train_idx]))
            x_train.append(np.apply_along_axis(calc_centroid, 2, nirs_data[train_idx]))
            
            x_train = np.concatenate(x_train, axis=1)
            
            x_test = []
            
            x_test.append(np.apply_along_axis(np.std, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(np.max, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(np.min, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(rms, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(slope, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(skew, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(np.average, 2, test_detail_coeffs[lvl]**2))
            x_test.append(np.apply_along_axis(mean_abs_diff, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(temp_centroid, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(energy, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(kurtosis, 2, test_detail_coeffs[lvl]))
            x_test.append(np.apply_along_axis(calc_centroid, 2, test_detail_coeffs[lvl]))

            x_test.append(np.apply_along_axis(np.std, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(np.max, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(np.min, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(rms, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(slope, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(skew, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(np.average, 2, test_detail_coeffs[lvl1]**2))
            x_test.append(np.apply_along_axis(mean_abs_diff, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(temp_centroid, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(energy, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(kurtosis, 2, test_detail_coeffs[lvl1]))
            x_test.append(np.apply_along_axis(calc_centroid, 2, test_detail_coeffs[lvl1]))

            x_test.append(np.apply_along_axis(np.std, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(np.max, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(np.min, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(rms, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(slope, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(skew, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(np.average, 2, test_aprox_coeffs[-1]**2))
            x_test.append(np.apply_along_axis(mean_abs_diff, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(temp_centroid, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(energy, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(kurtosis, 2, test_aprox_coeffs[-1]))
            x_test.append(np.apply_along_axis(calc_centroid, 2, test_aprox_coeffs[-1]))
            
            if fourth :
                x_test.append(np.apply_along_axis(np.std, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(np.max, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(np.min, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(rms, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(slope, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(skew, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(np.average, 2, test_detail_coeffs[fourth]**2))
                x_test.append(np.apply_along_axis(mean_abs_diff, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(temp_centroid, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(energy, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(kurtosis, 2, test_detail_coeffs[fourth]))
                x_test.append(np.apply_along_axis(calc_centroid, 2, test_detail_coeffs[fourth]))

            x_test.append(np.apply_along_axis(np.std, 2, nirs_data[test_idx]))
            x_test.append(np.apply_along_axis(np.mean, 2, nirs_data[test_idx]))
            x_test.append(np.apply_along_axis(np.min, 2, nirs_data[test_idx]))
            x_test.append(np.apply_along_axis(np.max, 2, nirs_data[test_idx]))
            x_test.append(np.apply_along_axis(slope, 2, nirs_data[test_idx]))
            x_test.append(np.apply_along_axis(skew, 2, nirs_data[test_idx]))
            x_test.append(np.apply_along_axis(kurtosis, 2, nirs_data[test_idx]))
            x_test.append(np.apply_along_axis(calc_centroid, 2, nirs_data[test_idx]))

            x_test = np.concatenate(x_test, axis=1)
            
            if mrmr_features is None:            
                x_pd = pd.DataFrame(x_train)
                mrmr_features = mrmr_classif(X=x_pd, y=y_train, K=features)
           
            lda.fit(x_train[:,mrmr_features], y_train)
            median_score.append(np.median(lda.score(x_test[:,mrmr_features], y_test)))

        selected_features.append(np.array(ft_names)[mrmr_features])
        score.append(np.median(median_score))
    
    print('sc:' , np.mean(score) , 'std', np.std(score))
    return selected_features, np.mean(score)